#### **Retrievers**  

Documentation - https://docs.langchain.com/oss/python/integrations/retrievers/index#retrievers  

A retriever is a component in LangChain that fetches relevant documents from a data source in response to a user query.  
Plug-and-play with chains because retrievers are Runnables (i.e., reusable & composable like models or prompts).  
Can be used in standalone chains or plugged into existing ones for more flexible RAG systems.  

**Types of Retrievers-** Retrievers can be categorized in two ways:  
**1. Based on Data Source:**  
Wikipedia Retriever – Fetches data directly from Wikipedia.  
Vector Store Retriever – Searches vector embeddings stored in vector DBs.  
Archive Retriever – Fetches academic papers from sites like Semantic Scholar, arXiv, etc.  

**2. Based on Search Strategy:**  
MMR (Max Marginal Relevance) – Balances relevance + diversity.  
Multi-Query Retriever – Generates and searches using multiple versions of the query.  
Contextual Compression Retriever – Compresses the context for efficiency.


##### **Based on Data Source**

##### **Wikipedia Retriever**  
The Wikipedia Retriever queries the Wikipedia API to fetch relevant articles matching the input query.  
It performs keyword-based search, not semantic or vector-based -> Internally uses keyword matching to determine article relevance.

In [2]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever(
    top_k_results=2,
    lang="en"
)

query = "The Geopolitical History of India and Pakistan from chinese perspective"

docs = retriever.invoke(query)

for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print("Page Content: ",doc.page_content)
    print("\n")

Document 1:
Page Content:  The United States has been providing military aid and economic assistance to Pakistan for various purposes since 1948. In 2017, the U.S. stopped military aid to Pakistan, which was about US$2 billion per year. With U.S. military assistance suspended in 2018 and civilian aid reduced to about $300 million for 2022, Pakistani authorities have turned to other countries for help.


== History ==
From 1947 to 1958, under civilian leadership, the United States provided Pakistan with modest economic aid and limited military assistance. During this period, Pakistan became a member of the South East Asian Treaty Organization (SEATO) and the Central Treaty Organization (CENTO), after a Mutual Defence Assistance Agreement signed in May 1954, which facilitated increased levels of both economic and military aid from the U.S.
In 1958, Ayub Khan led Pakistan's first military coup, becoming Chief Martial Law Administrator (CMLA) and later President until 1969. During his tenu

##### **Vector Score Retriever**  
Fetches relevant documents from a Vector Store using semantic similarity.  

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [ ]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"}
)

# In FAISS, we need to save manually 
vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embeddings,
)

# Converting Vector Store to Retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

query = "What is LangChain?"
results = retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"Document {i+1}:")
    print("Page Content: ", doc.page_content)

Document 1:
Page Content:  LangChain helps developers build LLM applications easily.
Document 2:
Page Content:  Chroma is a vector database optimized for LLM-based search.


##### **Based on Search Strategy**

##### **Maximum Marginal Relevance**  
Used to reduce redundancy and increase diversity in results.  
Fetches documents that are: Relevant to the query and Different from each other.  

**lambda_mult Parameter:** 
Controls balance between relevance and diversity:  
lambda=1: behaves like pure similarity search (less diverse).  
lambda=0: gives highly diverse results (may reduce relevance).  
Choose a value between 0 and 1 (e.g., 0.5) for best tradeoff.

In [14]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"}
)

vector_store = FAISS.from_documents(
    embedding=embeddings,
    documents=docs
)

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3, "lambda_mult": 0.4}  # k = top results, lambda_mult = relevance-diversity balance
)

query= "What is Langchain?"
results = retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"Result {i+1}:")
    print("Page Content: ", doc.page_content)

Result 1:
Page Content:  LangChain supports Chroma, FAISS, Pinecone, and more.
Result 2:
Page Content:  Embeddings are vector representations of text.
Result 3:
Page Content:  LangChain is used to build LLM based applications.


##### **Multi-Query Retriever**  
A special retriever in LangChain that:  
Expands one ambiguous query into multiple specific sub-queries using an LLM.  
Runs all sub-queries against the vector store.  
Merges and deduplicates the results.  
Returns the most relevant documents.

In [20]:
import sys
!{sys.executable} -m pip install --upgrade langchain

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.4
    Uninstalling langchain-core-1.0.4:
      Successfully uninstalled langchain-core-1.0.4
  Attempting uninstall: langchain
    Found existing installation: langchain 1.0.5
    Uninstalling langchain-1.0.5:
      Successfully uninstalled langchain-1.0.5



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from langchain_community.retrievers import MultiQueryRetriever
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()

documents = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"}
)

vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embeddings
)

# Create base retriever
base_retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})

# Create LLM (fixed: ChatGroq not Chatgroq, model not model_name)
llm = ChatGroq(model="llama-3.3-70b-versatile")

# Create MultiQueryRetriever
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever,
    llm=llm
)

query = "How to improve energy levels and maintain balance?"
multiquery_results = multiquery_retriever.invoke(query)

for i, doc in enumerate(multiquery_results):
    print(f"Result {i+1}:")
    print("Page Content: ", doc.page_content)
    print("\n")


ImportError: cannot import name 'MultiQueryRetriever' from 'langchain_community.retrievers' (c:\Users\Aryan\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\retrievers\__init__.py)

##### **Contextual Compression Retriever**  
Enhances retrieval quality by compressing documents after retrieval, keeping only the query-relevant parts.

Useful when documents are long or contain mixed/unrelated topics.

In [29]:
from langchain_core.retrievers import ContextualCompressionRetriever
from langchain_core.retrievers import LLMChainExtractor
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

ImportError: cannot import name 'ContextualCompressionRetriever' from 'langchain_core.retrievers' (c:\Users\Aryan\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\retrievers.py)

In [ ]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"}
)

vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)

base_retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 2})

llm = ChatGroq(model="llama-3.3-70b-versatile")
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)
